### Queries Quick Reference

You can generate the files using the queries listed in each analysis section, under the "Vehicle Events Analysis", "App Events Analysis", "Rental Event Analysis" sections. For more details, go to each individual section.

For reference:

- Vehicle Events Analysis Query (used for `VEHICLE_DATAFILE`)
```sql
SELECT from_iso8601_timestamp(timestamp) AS "event_datetime",
         "object_data-vin" as "vin",
         "object_data-is_available" as "is_available",
         "object_data-last_known_latitude" as "lat",
         "object_data-last_known_longitude" as "lng"
FROM "data_lake_us_prod"."sa_object_changed" -- region
WHERE "name" = 'VEHICLE_UPDATE'
        AND tenant_id = 'darwin-prod' -- customer
        AND timestamp > '2019-05-01' -- start date
        AND timestamp < '2019-05-15' -- end date
order by timestamp
```

- App Events Analysis (used for `APP_DATA_DATAFILE`)
```sql
select
from_iso8601_timestamp(timestamp) AS "event_datetime",
"data-distance",
"data-user_location-lat",
"data-user_location-lng",
"data-vehicle_latitude",
"data-vehicle_longitude"
from data_lake_us_prod.ma_user_activity -- region
where tenant_id = 'darwin-prod' -- customer
and name = 'DISTANCE_NEAREST_VEHICLE'
and timestamp > '2019-05-01' -- start date
and timestamp < '2019-05-15' -- end date
```

- Rental Events Analysis (used for `RENTAL_DATAFILE`)
```sql
SELECT "object_data-rental_id" as "rental_id",
        "object_data-customer_id" as "customer_id",
        "object_data-rental_reserved_at" as "reserved_at",
        "object_data-rental_booked_at" as "booked_at",
        "object_data-rental_ended_at" as "ended_at",
        "object_data-start_location_latitude" as "start_location_lat",
        "object_data-start_location_longitude" as "start_location_lng",
        "object_data-end_location_latitude" as "end_location_lat",
        "object_data-end_location_longitude" as "end_location_lng",
        payments.total_to_charge,
        payments.credit_amount_used
FROM (select distinct "object_data-rental_id" as "rental_id"
     from "data_lake_us_prod"."sa_object_changed" -- environment
     where "object_data-rental_reserved_at" > '2019-05-01' -- start date
     and "object_data-rental_reserved_at" < '2019-05-15' -- end date
    ) as rentals_list
JOIN "data_lake_us_prod"."sa_object_changed" -- environment
ON "object_data-rental_id" = rentals_list.rental_id
JOIN (select "object_data-rental_id" as "rental_id",
        "object_data-total_to_charge" as "total_to_charge",
        "object_data-credit_amount_used" as "credit_amount_used"
     from "data_lake_us_prod"."sa_object_changed" -- environment
     where tenant_id = 'darwin-prod' -- customer
     and name = 'PAYMENT_LIFECYCLE'
     group by "object_data-rental_id", "object_data-total_to_charge", "object_data-credit_amount_used"
     having "object_data-total_to_charge" is not null
    ) as payments 
ON payments.rental_id = rentals_list.rental_id
WHERE tenant_id = 'darwin-prod' -- customer
       AND name = 'RENTAL_LIFECYCLE'
       AND "object_data-rental_reserved_at" > '2019-05-01' -- start date
       AND "object_data-rental_reserved_at" < '2019-05-15' -- end date
GROUP BY
   "object_data-rental_id",
   "object_data-customer_id",
   "object_data-start_location_latitude",
   "object_data-start_location_longitude",
   "object_data-end_location_latitude",
   "object_data-end_location_longitude",
   "object_data-rental_reserved_at",
   "object_data-rental_booked_at",
   "object_data-rental_ended_at",
   payments.total_to_charge,
   payments.credit_amount_used
HAVING "object_data-start_location_latitude" is NOT null
       AND "object_data-start_location_longitude" is NOT null
       AND "object_data-end_location_latitude" is NOT null
       AND "object_data-end_location_longitude" is NOT null
       AND length(split(cast("object_data-end_location_latitude" as varchar), '.')[2]) > 4
       AND length(split(cast("object_data-end_location_longitude" as varchar), '.')[2]) > 4
       AND length(split(cast("object_data-start_location_latitude" as varchar), '.')[2]) > 4
       AND length(split(cast("object_data-start_location_longitude" as varchar), '.')[2]) > 4
```

- Alternative Rental Events Analysis (from prod db)
```sql
select 
    r.id as rental_id,
    r.customer_id as customer_id,
    r.start_datetime as reserved_at,
    r.booked_at as booked_at,
    r.end_datetime as ended_at,
    loc.lat as start_location_lat,
    loc.lng as start_location_lng,
    loc.postal_code,
    p.total_to_charge as total_to_charge,
    p.credit_used_amount as credit_amount_used
from carsharing_rental as r
    join common_location as loc on r.start_location_id = loc.id
    join carsharing_payment on carsharing_payment.rental_id = r.id
    join common_payment as p on p.id = carsharing_payment.payment_ptr_id
    join common_customer on common_customer.id = r.customer_id
    join common_userprofile on common_customer.user_profile_id = common_userprofile.id
    join auth_user on common_userprofile.user_id = auth_user.id
where auth_user.email not in (select email from common_paymentexemption)
    and auth_user.email not like '%ridecell.com'
    and auth_user.email not like '%eco-service.us'
    and auth_user.email not like '%ecoservice.us'
    and r.start_datetime > '2019-05-01'
    and r.start_datetime < '2019-06-06'
    and r.cancelled_at is null;
```

### Declare Files Below

These files will be retrieved by the notebook for analysis and processing prior to visualization.
After processing, new files will be generated with the naming convention "{filename}\_with\_dow\_hour\_mask.csv"

# Vehicle Events Analysis

The following query is to be used to retrieve data from Athena. 
The query collects vehicle all vehicle events between a given time for a given customer 
and retains the "is_available" field to determine vehicle availability (or idleness).

Change the following to suit your needs:
1. customer: code name for customer followed by environment with this format: '{customer}-{env}' example: 'darwin-prod'
2. start date: yyyy-mm-dd, inclusive. example: '2019-05-01'
3. end date: yyyy-mm-dd, exclusive. example: '2019-05-15'
4. region: 'data_lake_{region}_prod' where environment represents the region from where this data is pulled. Currently, we only support 'us' and 'eu'.
            
Query: 
```sql
SELECT from_iso8601_timestamp(timestamp) AS "event_datetime",
         "object_data-vin" as "vin",
         "object_data-is_available" as "is_available",
         "object_data-last_known_latitude" as "lat",
         "object_data-last_known_longitude" as "lng"
FROM "data_lake_us_prod"."sa_object_changed" -- region
WHERE "name" = 'VEHICLE_UPDATE'
        AND tenant_id = 'darwin-prod' -- customer
        AND timestamp > '2019-05-01' -- start date
        AND timestamp < '2019-05-15' -- end date
order by timestamp;
```

In [6]:
import calendar
from copy import deepcopy
import time

import pandas as pd
import numpy as np

from settings import region

VEHICLE_DATAFILE = 'vehicle_availability_data_eiffel_20190601_20190801.csv'
miles_per_meter = 0.000621371

In [7]:
mi = pd.MultiIndex.from_product([list(calendar.day_name), list(range(0, 24))], names=['dow', 'hour'])
base_series = pd.Series(index=mi).fillna(value=0)
mi_df = pd.DataFrame(columns=mi)
supply_df = pd.DataFrame()

### Convert timestamp to local region timezone

In [8]:
REGION_TIMEZONE = region['oakland']['timezone']

def convert_datetime_columns(df, columns):
    for col in columns:
        try:
            df[col] = df[col].dt.tz_localize('UTC').dt.tz_convert(REGION_TIMEZONE)
        except TypeError:
            df[col] = df[col].dt.tz_convert(
                   'UTC').dt.tz_convert(REGION_TIMEZONE)

### Compress repeat events

This function collapses repeated "is_available" events and creates two new columns 
"available_at" and "unavailable_at", representing a range, where "available_at" comes before "unavailable_at". 

In [9]:
def collapse_is_available_events(group):
    global supply_df
    group = group.sort_values(by='event_datetime')

    # get time of change of states

    # get event_datetime when is_available goes from true to false (becomes unavailable)
    # previous event (is_available=True) changed state (is_available=False), indicating becoming unavailable
    left = group[(group['is_available'] == False) & (group['is_available'].shift() == True)].rename(
        columns={'event_datetime':'unavailable_at'})

    # get event_datetime when is_available goes from false to true (becomes available)
    # previous event (is_available=False) changed state (is_available=True), indicating becoming available
    right = group[(group['is_available'] == True) & (group['is_available'].shift() == False)].rename(
        columns={'event_datetime':'available_at'})['available_at'].to_frame()

    # can't assume symmetry for events
    # can't tell which event comes first
    merged_group = pd.merge_asof(left, right, left_on='unavailable_at', right_on='available_at')
    supply_df = supply_df.append(merged_group)

    global vehicle_df_vin_grouped

    if not supply_df.shape[0] % 1000:
        print(f'{supply_df.shape[0]} events collapsed')

### Extract dow/hour from range of times

This function takes a range of datetime indices and extracts dow and hour and places it into a 2d matrix 
(represented in dataframes by a multi-index).

In [10]:
# construct large dow/hour df
# NOTE: very expensive. should save intermediates so don't have to regenerate
def extractor(x):
    global mi_df
    temp = deepcopy(base_series)
    # duration less than 1 hour, does span across slice (hour) ex: [1:30, 2:15]
    if x.size == 2 and x[0].hour != x[1].hour:
        temp[x[0].day_name(), x[0].hour] += 60 - x[0].minute
        temp[x[1].day_name(), x[1].hour] += x[1].minute

    # duration less than 1 hour, doesn't span across slice (hour) ex: [1:30, 1:45]
    elif x.size == 2 and x[0].hour == x[1].hour:
        temp[x[0].day_name(), x[0].hour] += x[1].minute - x[0].minute

    # duration greater than 1 hour, does span across slice (hour) ex: [1:30, 2:30, 2:45]
    elif x.size == 3 and x[1].hour == x[2].hour:
        temp[x[0].day_name(), x[0].hour] += 60 - x[0].minute
        temp[x[2].day_name(), x[2].hour] += x[2].minute

    else:
        # duration greater than 2 hours, ex: [1:30, 2:30, 3:30, 3:45]
        # or spans across multiple hours
        n = 0
        min_marker = x[0].minute
        for i, j, k in zip(x.day_name(), x.hour, x.minute):
            # each datetimeindex
            if n == 0: # first element => 60 - 30 = 30
                temp[i, j] += (60 - k)
            elif n == (x.size - 1):  # last element, can't assume full hour
                if k >= min_marker:
                    temp[i, j] += (k - min_marker) # ex: 3:45 - 3:30 = 15m
                else:
                    temp[i, j] += k  # ex: 3:30 - 3:00 = 30m
            elif n == (x.size - 2):  # second to last element, can't assume full hour
                temp[i, j] += k  # ex: 3:30 - 3:00 = 30m
            else:  # middle of array
                temp[i, j] += 60 # ex: 3:30 - 2:30 = 1h
            n += 1
    mi_df = mi_df.append(temp, ignore_index=True)
    # get size incoming vehicle events
    global df
    # determine size of mi_df
    # report every 10000 events
    if not mi_df.shape[0] % 10000:
        print(f'mask {mi_df.shape[0]/df.shape[0] * 100}% complete')

### Generate Vehicle Event Analysis Intermediates

Uses the above functions in order to reduce and extract idle vehicle minutes to be mapped onto a 2d matrix representing dow and hour of the day. Intakes a file and outputs a processed file to be filtered for visualization.

In [ ]:
VEHICLE_DT_COLS = ['event_datetime']

print('Loading file...')
read_init_time = time.time()

# get df and clean up
vehicle_df = pd.read_csv(
    VEHICLE_DATAFILE,
    parse_dates=VEHICLE_DT_COLS,
    infer_datetime_format=True
).dropna()
print('Done loading file...')
print(f'Took {int(time.time() - read_init_time)}s')

print('Converting datetime columns...')
convert_datetime_columns(vehicle_df, VEHICLE_DT_COLS)
print('Done converting datetime columns...')

# group by vin
vehicle_df_vin_grouped = vehicle_df.groupby(['vin'])

print('Collapsing events...')
collapse_init_time = time.time()
vehicle_df_vin_grouped.apply(collapse_is_available_events)
print(f'Collapse time: {time.time() - collapse_init_time}s')

supply_df = supply_df.dropna()
supply_df.reset_index(inplace=True)
supply_df['idle_duration'] = supply_df['unavailable_at'] - supply_df['available_at']  # duration for analysis
supply_df['idle_duration_minutes'] = supply_df['idle_duration'].dt.total_seconds()/60.0

# create datetimeindex of periods with the end datetime appended
df = supply_df.apply(
    lambda x: (pd.date_range(x['available_at'], x['unavailable_at'], freq='H', closed='left')).append(
        pd.to_datetime([x['unavailable_at']])), axis=1)

print('Extracting events...')
extract_init_time = time.time()
df.apply(extractor)
print(f'Extract time: {time.time() - extract_init_time}s')

# merge the big dow/hour mask back with vehicle_update data
supply_df = supply_df.merge(mi_df, left_index=True, right_index=True)
supply_df.to_csv(f'{VEHICLE_DATAFILE.split(".")[0]}_with_dow_hour_mask.csv')

Loading file...
Done loading file...
Took 1234s
Converting datetime columns...
Done converting datetime columns...
Collapsing events...


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Collapse time: 51.43484020233154s
Extracting events...
mask 24.078109386850944% complete
mask 26.48592032553604% complete
mask 28.893731264221135% complete
mask 31.30154220290623% complete
mask 33.70935314159132% complete
mask 36.11716408027642% complete
mask 38.52497501896151% complete
mask 40.93278595764661% complete
mask 43.340596896331704% complete
mask 45.7484078350168% complete
mask 48.15621877370189% complete
mask 50.56402971238698% complete
mask 52.97184065107208% complete
mask 55.379651589757174% complete
mask 57.78746252844227% complete
mask 60.195273467127365% complete
mask 62.60308440581246% complete
mask 65.01089534449756% complete
mask 67.41870628318264% complete
mask 69.82651722186773% complete
mask 72.23432816055283% complete
mask 74.64213909923792% complete
mask 77.04995003792303% complete
mask 79.45776097660811% complete
mask 81.86557191529322% complete
mask 84.2733828539783% complete
mask 86.68119379266341% complete
mask 89.0890047313485% complete
mask 91.49681567003

In [ ]:
supply_df.head()

# App Events Analysis

We load the data, clean up timestamps, and add columns representing the following:
1. vehicle_nearby: a boolean representing if there was a vehicle within a distance threshold at the time of the app event
2. event_hour: extracted hour of app event time in 24H time
3. event_dow: extracted day of the week of app event time, as Monday, Tuesday, etc.

Change the following parameters of the query to suit your needs:
- customer: code name for customer followed by environment with this format: '{customer}-{env}' example: 'darwin-prod'
- start date: yyyy-mm-dd, inclusive. example: '2019-05-01'
- end date: yyyy-mm-dd, exclusive. example: '2019-05-15'
- region: 'data_lake_{region}_prod' where environment represents the region from where this data is pulled. Currently, we only support 'us' and 'eu'.
       
Events are pulled from Athena (datalake) using this query:
```sql
select
from_iso8601_timestamp(timestamp) AS "event_datetime",
"data-distance",
"data-user_location-lat",
"data-user_location-lng",
"data-vehicle_latitude",
"data-vehicle_longitude"
from data_lake_us_prod.ma_user_activity -- region
where tenant_id = 'darwin-prod' -- customer
and name = 'DISTANCE_NEAREST_VEHICLE'
and timestamp > '2019-05-01' -- start date
and timestamp < '2019-05-15'; -- end date
```

For eiffel, the query changes a bit:

```sql
select
from_iso8601_timestamp(timestamp) AS "event_datetime",
"data-closest_vehicle-distance_to_user",
"data-customer-lat",
"data-customer-lng",
"data-closest_vehicle-lat",
"data-closest_vehicle-lng"
from data_lake_eu_prod.ma_user_activity -- region
where name = 'VEHICLE_AVAILABILITY'
AND tenant_id = 'eiffel-prod' -- customer
and timestamp >= '2019-06-01'
and timestamp < '2019-08-01'
order by timestamp desc
```

In [4]:
# app events constants
rental_distance_threshold = 0.25
APP_DATA_DATAFILE = 'app_event_data_eiffel_20190601_20190801.csv'

In [5]:
import pandas as pd

APP_DATA_DT_COLS = ['event_datetime']
app_df = pd.read_csv(
        APP_DATA_DATAFILE,
        parse_dates=APP_DATA_DT_COLS,
        infer_datetime_format=True
    ).dropna()

convert_datetime_columns(app_df, APP_DATA_DT_COLS)

app_df.columns = ['event_datetime', 'data-distance', 'data-user_location-lat',
                  'data-user_location-lng', 'data-vehicle_latitude', 'data-vehicle_longitude']

# create new column to test if within threshold
app_df['vehicle_nearby'] = (app_df['data-distance'] * miles_per_meter) < rental_distance_threshold

app_df['event_hour'] = app_df['event_datetime'].dt.hour
app_df['event_dow'] = app_df['event_datetime'].dt.day_name()

app_df.to_csv(f'{APP_DATA_DATAFILE.split(".")[0]}_with_threshold_mask.csv')

NameError: name 'convert_datetime_columns' is not defined

# Rental Events Analysis
We load the data, clean up timestamps, and add columns representing the following:
1. reserved_at_hour: extracted hour of app event time in 24H time
2. reserved_at_dow: extracted day of the week of app event time, as Monday, Tuesday, etc.
    
Change the following parameters of the query to suit your needs:
- customer: code name for customer followed by environment with this format: '{customer}-{env}' example: 'darwin-prod'
- start date: yyyy-mm-dd, inclusive. example: '2019-05-01'
- end date: yyyy-mm-dd, exclusive. example: '2019-05-15'
- environment: 'data_lake_{environment}_prod' where environment represents the region from where this data is pulled. Currently, we only support 'us' and 'eu'.
    
Events are pulled from Athena (datalake) using this query:

```sql
SELECT "object_data-rental_id" as "rental_id",
          "object_data-customer_id" as "customer_id",
          "object_data-rental_reserved_at" as "reserved_at",
          "object_data-rental_booked_at" as "booked_at",
          "object_data-rental_ended_at" as "ended_at",
          "object_data-start_location_latitude" as "start_location_lat",
          "object_data-start_location_longitude" as "start_location_lng",
          "object_data-end_location_latitude" as "end_location_lat",
          "object_data-end_location_longitude" as "end_location_lng",
          payments.total_to_charge,
          payments.credit_amount_used
 FROM (select distinct "object_data-rental_id" as "rental_id"
       from "data_lake_us_prod"."sa_object_changed" -- environment
       where "object_data-rental_reserved_at" > '2019-05-01' -- start date
       and "object_data-rental_reserved_at" < '2019-05-02' -- end date
      ) as rentals_list
 JOIN "data_lake_us_prod"."sa_object_changed" -- environment
 ON "object_data-rental_id" = rentals_list.rental_id
 JOIN (select "object_data-rental_id" as "rental_id",
          "object_data-total_to_charge" as "total_to_charge",
          "object_data-credit_amount_used" as "credit_amount_used"
       from "data_lake_us_prod"."sa_object_changed" -- environment
       where tenant_id = 'darwin-prod' -- customer
       and name = 'PAYMENT_LIFECYCLE'
       group by "object_data-rental_id", "object_data-total_to_charge", "object_data-credit_amount_used"
       having "object_data-total_to_charge" is not null
      ) as payments 
 ON payments.rental_id = rentals_list.rental_id
 WHERE tenant_id = 'darwin-prod' -- customer
         AND name = 'RENTAL_LIFECYCLE'
 GROUP BY
     "object_data-rental_id",
     "object_data-customer_id",
     "object_data-start_location_latitude",
     "object_data-start_location_longitude",
     "object_data-end_location_latitude",
     "object_data-end_location_longitude",
     "object_data-rental_reserved_at",
     "object_data-rental_booked_at",
     "object_data-rental_ended_at",
     payments.total_to_charge,
     payments.credit_amount_used
 HAVING "object_data-start_location_latitude" is NOT null
         AND "object_data-start_location_longitude" is NOT null
         AND "object_data-end_location_latitude" is NOT null
         AND "object_data-end_location_longitude" is NOT null
         AND length(split(cast("object_data-end_location_latitude" as varchar), '.')[2]) > 4
         AND length(split(cast("object_data-end_location_longitude" as varchar), '.')[2]) > 4
         AND length(split(cast("object_data-start_location_latitude" as varchar), '.')[2]) > 4
         AND length(split(cast("object_data-start_location_longitude" as varchar), '.')[2]) > 4
```

for eiffel from db:
```sql
select 
  r.id as rental_id,
  r.customer_id as customer_id,
  r.start_datetime as reserved_at,
  r.booked_at as booked_at,
  r.end_datetime as ended_at,
  loc.lat as start_location_lat,
  loc.lng as start_location_lng,
  loc.postal_code,
  p.total_to_charge as total_to_charge,
  p.credit_used_amount as credit_amount_used
from carsharing_rental as r
  join common_location as loc on r.start_location_id = loc.id
  join carsharing_payment on carsharing_payment.rental_id = r.id
  join common_payment as p on p.id = carsharing_payment.payment_ptr_id
  join common_customer on common_customer.id = r.customer_id
  join common_userprofile on common_customer.user_profile_id = common_userprofile.id
  join auth_user on common_userprofile.user_id = auth_user.id
where auth_user.email not in (select email from common_paymentexemption)
  and auth_user.email not like '%ridecell.com'
  and auth_user.email not like '%zity%'
  and auth_user.email not like '%ferrovial%'
  and auth_user.email not like '%renault%'
  and r.start_datetime >= '2019-06-01'
  and r.start_datetime < '2019-08-01'
  and r.cancelled_at is null;
```

In [ ]:
# get the rentals file
RENTAL_DATAFILE = 'rental_data_eiffel_db_20190601_20190801.csv'
RENTAL_DT_COLS = ['reserved_at', 'booked_at', 'ended_at']

# get df and clean up
rental_df = pd.read_csv(
    RENTAL_DATAFILE,
    parse_dates=RENTAL_DT_COLS,
    infer_datetime_format=True
).fillna(0)

convert_datetime_columns(rental_df, RENTAL_DT_COLS)

# extract the rental start dow/hour
rental_df['reserved_at_hour'] = rental_df['reserved_at'].dt.hour
rental_df['reserved_at_dow'] = rental_df['reserved_at'].dt.day_name()

rental_df.to_csv(f'{RENTAL_DATAFILE.split(".")[0]}_with_dow_hour_mask.csv')